# Azure AI Document Intelligence - Prebuilt Models

Dieses Notebook zeigt, wie man vorgefertigte Modelle von Azure AI Document Intelligence verwendet:
- **Invoice Model** - Rechnungen analysieren
- **Receipt Model** - Quittungen analysieren
- **ID Document Model** - Ausweisdokumente analysieren
- **Business Card Model** - Visitenkarten analysieren

## 1. Installation

In [11]:
# Pakete installieren
!uv pip install python-dotenv azure-ai-formrecognizer azure-storage-blob ipywidgets

Using Python 3.13.5 environment at: /Users/oscharko/PycharmProjects/Keiko-Evolutio/demo-it-tage-2025/.venv
Audited 4 packages in 3ms


## 2. Umgebungsvariablen laden

In [12]:
import os
from pathlib import Path
from dotenv import load_dotenv

env_path = Path('..') / '..' / '.env'
load_dotenv(env_path)

print('Konfiguration geladen:')
print(f"  • Document Intelligence Endpoint: {os.getenv('DOCUMENT_INTELLIGENCE_ENDPOINT')}")
print(f"  • Storage Container: {os.getenv('DOC_INTEL_CONTAINER_NAME')}")

Konfiguration geladen:
  • Document Intelligence Endpoint: https://doc-intel-doc-intelligence-it-tage-2025.cognitiveservices.azure.com/
  • Storage Container: sample-documents


## 3. Clients initialisieren

In [13]:
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient

# Document Intelligence Client
endpoint = os.getenv('DOCUMENT_INTELLIGENCE_ENDPOINT')
api_key = os.getenv('DOCUMENT_INTELLIGENCE_API_KEY')

if not endpoint or not api_key:
    raise ValueError('DOCUMENT_INTELLIGENCE_ENDPOINT und DOCUMENT_INTELLIGENCE_API_KEY müssen in .env gesetzt sein')

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(api_key)
)

# Blob Storage Client
storage_connection_string = os.getenv('DOC_INTEL_STORAGE_CONNECTION_STRING')
container_name = os.getenv('DOC_INTEL_CONTAINER_NAME')

if not storage_connection_string or not container_name:
    raise ValueError('DOC_INTEL_STORAGE_CONNECTION_STRING und DOC_INTEL_CONTAINER_NAME müssen in .env gesetzt sein')

blob_service_client = BlobServiceClient.from_connection_string(storage_connection_string)
container_client = blob_service_client.get_container_client(container_name)

print('✓ Clients initialisiert!')

✓ Clients initialisiert!


## 4. Verfügbare Dokumente auflisten

In [14]:
print('Dokumente im Blob Storage:')
print('=' * 80)

blobs = list(container_client.list_blobs())

if not blobs:
    print('⚠️  Keine Dokumente gefunden!')
    print('   Bitte lade zuerst Dokumente hoch.')
else:
    for idx, blob in enumerate(blobs, start=1):
        size_kb = blob.size / 1024
        print(f"{idx}. {blob.name} ({size_kb:.2f} KB)")

Dokumente im Blob Storage:
1. 01.png (1548.92 KB)
2. 02.png (268.69 KB)
3. 03.png (2148.54 KB)
4. 04.png (425.30 KB)
5. beispiel-rechnung-01.png (649.73 KB)


## 5. Hilfsfunktion: Dokument analysieren

In [15]:
def analyze_document(blob_name, model_id):
    """Analysiere ein Dokument mit einem bestimmten Modell"""
    print(f'Analysiere: {blob_name}')
    print(f'Modell: {model_id}')
    print('=' * 80)
    
    # Hole Blob URL
    blob_client = container_client.get_blob_client(blob_name)
    blob_url = blob_client.url
    
    # Analysiere Dokument
    try:
        poller = document_analysis_client.begin_analyze_document_from_url(
            model_id=model_id,
            document_url=blob_url
        )
        
        result = poller.result()
        print('✓ Analyse abgeschlossen!')
        return result
    except Exception as e:
        print(f'✗ Fehler: {e}')
        return None

## 6. Invoice Model (Rechnungen)

Das Invoice Model extrahiert strukturierte Daten aus Rechnungen.

In [16]:
# Suche nach einer Rechnung
invoice_blobs = [b for b in blobs if 'invoice' in b.name.lower() or 'rechnung' in b.name.lower()]

if invoice_blobs:
    result = analyze_document(invoice_blobs[0].name, "prebuilt-invoice")
    
    if result and result.documents:
        print()
        for idx, invoice in enumerate(result.documents, start=1):
            print(f'Rechnung {idx}:')
            print('-' * 80)
            
            fields = invoice.fields
            
            if 'VendorName' in fields and fields['VendorName'].value:
                print(f'Lieferant: {fields["VendorName"].value}')
            
            if 'InvoiceId' in fields and fields['InvoiceId'].value:
                print(f'Rechnungsnummer: {fields["InvoiceId"].value}')
            
            if 'InvoiceDate' in fields and fields['InvoiceDate'].value:
                print(f'Rechnungsdatum: {fields["InvoiceDate"].value}')
            
            if 'InvoiceTotal' in fields and fields['InvoiceTotal'].value:
                total = fields['InvoiceTotal'].value
                if hasattr(total, 'amount'):
                    currency = total.currency_code if hasattr(total, 'currency_code') else ''
                    print(f'Gesamtbetrag: {total.amount} {currency}')
                else:
                    print(f'Gesamtbetrag: {total}')
else:
    print('Keine Rechnung gefunden.')
    print('Tipp: Benenne eine Datei mit "invoice" oder "rechnung" im Namen.')

Analysiere: beispiel-rechnung-01.png
Modell: prebuilt-invoice
✓ Analyse abgeschlossen!

Rechnung 1:
--------------------------------------------------------------------------------
Lieferant: MM
Max Mustermann GmbH
Rechnungsnummer: Re-172/2025
Rechnungsdatum: 2025-04-29
Gesamtbetrag: 868.97 


## 7. Receipt Model (Quittungen)

Das Receipt Model extrahiert Daten aus Kassenbelegen.

In [17]:
# Suche nach einer Quittung
receipt_blobs = [b for b in blobs if 'receipt' in b.name.lower() or 'quittung' in b.name.lower()]

if receipt_blobs:
    result = analyze_document(receipt_blobs[0].name, "prebuilt-receipt")
    
    if result and result.documents:
        print()
        for idx, receipt in enumerate(result.documents, start=1):
            print(f'Quittung {idx}:')
            print('-' * 80)
            
            fields = receipt.fields
            
            if 'MerchantName' in fields and fields['MerchantName'].value:
                print(f'Händler: {fields["MerchantName"].value}')
            
            if 'TransactionDate' in fields and fields['TransactionDate'].value:
                print(f'Datum: {fields["TransactionDate"].value}')
            
            if 'Total' in fields and fields['Total'].value:
                total = fields['Total'].value
                if hasattr(total, 'amount'):
                    print(f'Gesamtbetrag: {total.amount}')
                else:
                    print(f'Gesamtbetrag: {total}')
else:
    print('Keine Quittung gefunden.')
    print('Tipp: Benenne eine Datei mit "receipt" oder "quittung" im Namen.')

Keine Quittung gefunden.
Tipp: Benenne eine Datei mit "receipt" oder "quittung" im Namen.


## 8. ID Document Model (Ausweisdokumente)

Das ID Document Model extrahiert Daten aus Personalausweisen, Reisepässen, etc.

In [18]:
# Suche nach einem Ausweisdokument
id_blobs = [b for b in blobs if 'id' in b.name.lower() or 'passport' in b.name.lower() or 'ausweis' in b.name.lower()]

if id_blobs:
    result = analyze_document(id_blobs[0].name, "prebuilt-idDocument")
    
    if result and result.documents:
        print()
        for idx, id_doc in enumerate(result.documents, start=1):
            print(f'Ausweisdokument {idx}:')
            print('-' * 80)
            
            fields = id_doc.fields
            
            if 'FirstName' in fields and fields['FirstName'].value:
                print(f'Vorname: {fields["FirstName"].value}')
            
            if 'LastName' in fields and fields['LastName'].value:
                print(f'Nachname: {fields["LastName"].value}')
            
            if 'DateOfBirth' in fields and fields['DateOfBirth'].value:
                print(f'Geburtsdatum: {fields["DateOfBirth"].value}')
            
            if 'DocumentNumber' in fields and fields['DocumentNumber'].value:
                print(f'Dokumentennummer: {fields["DocumentNumber"].value}')
else:
    print('Kein Ausweisdokument gefunden.')
    print('Tipp: Benenne eine Datei mit "id", "passport" oder "ausweis" im Namen.')

Kein Ausweisdokument gefunden.
Tipp: Benenne eine Datei mit "id", "passport" oder "ausweis" im Namen.


## 9. Business Card Model (Visitenkarten)

Das Business Card Model extrahiert Kontaktdaten von Visitenkarten.

In [19]:
# Suche nach einer Visitenkarte
card_blobs = [b for b in blobs if 'card' in b.name.lower() or 'visitenkarte' in b.name.lower()]

if card_blobs:
    result = analyze_document(card_blobs[0].name, "prebuilt-businessCard")
    
    if result and result.documents:
        print()
        for idx, card in enumerate(result.documents, start=1):
            print(f'Visitenkarte {idx}:')
            print('-' * 80)
            
            fields = card.fields
            
            if 'ContactNames' in fields and fields['ContactNames'].value:
                for name in fields['ContactNames'].value:
                    first = name.value.get('FirstName', {}).value if 'FirstName' in name.value else ''
                    last = name.value.get('LastName', {}).value if 'LastName' in name.value else ''
                    print(f'Name: {first} {last}')
            
            if 'CompanyNames' in fields and fields['CompanyNames'].value:
                for company in fields['CompanyNames'].value:
                    print(f'Firma: {company.value}')
            
            if 'Emails' in fields and fields['Emails'].value:
                for email in fields['Emails'].value:
                    print(f'E-Mail: {email.value}')
else:
    print('Keine Visitenkarte gefunden.')
    print('Tipp: Benenne eine Datei mit "card" oder "visitenkarte" im Namen.')

Keine Visitenkarte gefunden.
Tipp: Benenne eine Datei mit "card" oder "visitenkarte" im Namen.


## 10. Zusammenfassung

In [20]:
print('Verfügbare Prebuilt Models:')
print('=' * 80)
print('✓ prebuilt-invoice      - Rechnungen')
print('✓ prebuilt-receipt      - Quittungen')
print('✓ prebuilt-idDocument   - Ausweisdokumente')
print('✓ prebuilt-businessCard - Visitenkarten')
print('✓ prebuilt-layout       - Allgemeines Layout')
print()
print('Weitere Modelle:')
print('  • prebuilt-read       - OCR für Text')
print('  • prebuilt-document   - Allgemeine Dokumente')
print('  • prebuilt-tax.us.w2  - US W-2 Steuerformulare')
print()
print('Dokumentation:')
print('https://learn.microsoft.com/azure/ai-services/document-intelligence/concept-model-overview')

Verfügbare Prebuilt Models:
✓ prebuilt-invoice      - Rechnungen
✓ prebuilt-receipt      - Quittungen
✓ prebuilt-idDocument   - Ausweisdokumente
✓ prebuilt-businessCard - Visitenkarten
✓ prebuilt-layout       - Allgemeines Layout

Weitere Modelle:
  • prebuilt-read       - OCR für Text
  • prebuilt-document   - Allgemeine Dokumente
  • prebuilt-tax.us.w2  - US W-2 Steuerformulare

Dokumentation:
https://learn.microsoft.com/azure/ai-services/document-intelligence/concept-model-overview
